<a href="https://colab.research.google.com/github/DartDoesData/python-practice/blob/main/Day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
# Preview the DataFrame
sr_311_gdf = gpd.read_file('https://services1.arcgis.com/UWYHeuuJISiGmgXx/arcgis/rest/services/311_Customer_Service_Requests_2024/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
sr_311_gdf.head()

In [ ]:
# Define the path for open 311 service requests
sr_311_path = 'https://docs.google.com/spreadsheets/d/1ptZy6JonuS5xHPRfDeAoEiz6PcpWu4_ZI1OcPvfg1GA/gviz/tq?tqx=out:csv'

# Read in the 311 service requests
sr_311_gdf = pd.read_csv(sr_311_path)

In [ ]:
open_311_sr_gdf = sr_311_gdf[~sr_311_gdf['SRStatus'].str.contains('Duplicate|Closed', na=False)]
open_311_sr_gdf.columns

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Baltimore coordinates
baltimore_coords = [39.2904, -76.6122]

# Create a map centered on Baltimore
map_baltimore = folium.Map(location=baltimore_coords, zoom_start=11)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(map_baltimore)

# Iterate through the DataFrame and add markers for each service request
for index, row in open_311_sr_gdf.iterrows():
    try:
        latitude = row['Latitude']
        longitude = row['Longitude']
        if not pd.isnull(latitude) and not pd.isnull(longitude):
            folium.Marker(
                location=[latitude, longitude],
                popup=row['SRType']
            ).add_to(marker_cluster)
    except KeyError:
        print("Latitude or Longitude column not found in DataFrame")

# Display the map
map_baltimore

In [ ]:
import plotly.express as px

# Group by SRType and count the number of requests
sr_type_counts = open_311_sr_gdf.groupby('SRType')['SRType'].count()

# Sort the counts in descending order and select the top 10
top_10_sr_types = sr_type_counts.sort_values(ascending=False).head(10)

# Create a horizontal bar chart using Plotly Express
fig = px.bar(
    x=top_10_sr_types.values,
    y=top_10_sr_types.index,
    orientation='h',
    labels={'x': 'Number of Service Requests', 'y': 'Service Request Type'},
    title='Top 10 Service Request Types in Baltimore'
)

# Update layout to show data labels
fig.update_traces(texttemplate='%{x}', textposition='outside')
fig.update_layout(
    xaxis_title='Number of Service Requests',
    yaxis_title='Service Request Type'
)

# Reverse the order of bars to put the highest count at the top
fig.update_layout(yaxis={'categoryorder':'total ascending'})

fig.show()

In [ ]:
import geopandas as gpd

neighborhoods_gdf = gpd.read_file('https://geodata.baltimorecity.gov/egis/rest/services/CityView/Neighborhoods/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
neighborhoods_gdf.head()

In [ ]:
# Join neighborhoods_gdf with open_311_sr_gdf to get a view of service requests by neighborhood